In [4]:
import pandas as pd
from pathlib import Path
import datetime
from datetime import time

Needs:
x Create df for one user
save night points to a list
save day points to a list
cluster these points
home will be cluster with most day points
work will be cluster with most night points

convert from gmt to local time using gps coords

In [8]:
user_id = '000'
data_path = Path('../data/raw')
user_path = data_path / user_id / 'Trajectory'

def read_plt_file(filepath):
    """Read a single PLT trajectory file"""
    df = pd.read_csv(
        filepath,
        skiprows=6,
        header=None,
        names=['lat', 'lon', 'zero', 'altitude', 'days', 'date', 'time']
    )
    
    # Combine date and time into datetime
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])

    df['hour'] = df['datetime'].dt.hour

    # Add a column to add name of trajectory file
    df['filename'] = filepath.name

    # Add user name
    df['user'] = user_id
    
    # Drop unnecessary columns
    df = df[['user', 'lat', 'lon', 'altitude', 'datetime', 'hour', 'filename']]
    
    return df

# Get their trajectory files
trajectory_files = list(user_path.glob('*.plt'))
all_traj_files = []

for traj_file in trajectory_files:
    traj = read_plt_file(traj_file)
    all_traj_files.append(traj)

df = pd.concat(all_traj_files, ignore_index=True)


Save night points and day points to df
all times are saved in gmt, beijing time is gmt + 8
want to implement a way later to convert time according to gps coords

night points in gmt: 23:00 to 7:00
day points in gmt: 9:00 to 17:00

subtracting 8 hrs for beijing time
night points: 15:00 to 23:00
day points: 1:00 to 9:00

In [6]:
print(type(df['datetime'][0]))

<class 'pandas.Timestamp'>


Want to add that day points are only for mon-fri. What I have below is for 
all 7 days

In [14]:

night_points = df[df['hour'].between(15, 23)]
day_points = df[df['hour'].between(1, 9)]

use hdbscan to cluster 

In [15]:
import hdbscan

clusterer = hdbscan.HDBSCAN(min_cluster_size=5)

coords = night_points[['lat', 'lon']].values
clusters = clusterer.fit_predict(coords)

In [16]:
night_points['cluster'] = clusters

# Count points per cluster (exclude -1 which is noise)
valid_clusters = night_points[night_points['cluster'] != -1]
home_cluster = valid_clusters['cluster'].value_counts().idxmax()

home_location = night_points[night_points['cluster'] == home_cluster][['lat', 'lon']].mean()

In [17]:
print(f"Home location: {home_location['lat']}, {home_location['lon']}")

Home location: 40.004885519163764, 116.30801704878051
